# Numerical Methods

## Numerical Linear Algebra II

In [14]:
import numpy as np
import scipy.linalg as sl

## Learning objectives:

* More on *direct* linear solver methods: the LU decomposition of a matrix


* Doolittle's algorithm


* Properties of lower-triangular matrices


* Partial pivoting

* Introduce ill-conditioned matrices (via matrix norms and the condition number)



## Introduction

Last week we developed and implemented the *Gaussian elimination* method to solve the linear matrix system 

$A\boldsymbol{x}=\boldsymbol{b}$. 

This week we will consider a closely related solution method: *LU decomposition* or *LU factorisation*.

Both are examples of *direct* solution methods - in the next lecture we will consider the alternative approach to solve linear systems, namely *iterative* or *indirect* methods.

## LU decomposition - motivation

Last week we implemented Gaussian elimination to solve the matrix system with *one* RHS vector $\boldsymbol{b}$.  

It is often that case that we have to deal with problems where we have multiple RHS vectors, all with the same matrix $A$.

We could call the same code (e.g. our 'upper_triangle' and 'back_substitution' codes from last lecture) multiple times to solve all of these corresponding linear systems, but note that as the elimination algorithm is actually performing operations based upon (the same) $A$ each time, we would be wasting time repeating exactly the same operations over and over again - this is therefore clearly not an efficient solution to this problem.

We could easily generalise our Gaussian elimination/back substitution algorithms to include multiple RHS column vectors in the augmented system, perform the same sequence of row operations (but now only once) to transform the matrix to upper-triangular form, and then perform back substitution on each of the transformed RHS vectors from the augmented system - cf. the use of Gaussian elimination to compute the inverse to a matrix by placing the identity matrix (or equivalently the columns of the identity matrix) on the right of the augmented system: $[A|\, \textrm{col}_1,\, \textrm{col}_2, \ldots]$.

However, it is often the case that each RHS vector depends on the solutions to the matrix systems obtained from some or all of the earlier RHS vectors, and so this generalisation would not work in this case. For example, our discrete system could be of the form

$$A\boldsymbol{x}^{n+1} = \boldsymbol{b}(\boldsymbol{x}^n, \ldots)$$

where $\boldsymbol{x}^{n+1}$ is the numerical solution of a (ordinary or partial) differential equation at time level $n+1$, the RHS is a function of the the solution at the previous time level $n$ (and possibly other things such as forcing terms, represented by the "$\ldots$"). $A$ is then a discretisation of the differential equation written in the form that it maps the old solution to the new one. Time stepping this problem involves solving the same linear system multiple times, but with different RHSs. 

[NB. this is true if our model is *linear*, in the nonlinear case $A$ would be updated every time step].

## LU decomposition - theory

To deal with this situation efficiently we *decompose* or *factorise* the matrix $A$ in such a way that it is cheap to compute a new solution vector $\boldsymbol{x}$ for any given RHS vector $\boldsymbol{b}$.  This decomposition involves a lower- and an upper-triangular matrix, hence the name LU decomposition. These matrices essentially *encode* the steps conducted in Gaussian elimination, so we don't have to explicilty conduct all of the operations again and again.

Mathematically, let's assume that we have already found/constructed a lower-triangular matrix ($L$ - where all entries above the diagonal are zero) and an upper-triangular matrix ($U$ - where all entries below the diagonal are zero) such that we can write

$$ A = LU $$

In this case the matrix system we need to solve for $\boldsymbol{x}$ becomes

$$ A\boldsymbol{x} = \boldsymbol{b} \iff (LU)\boldsymbol{x} = \boldsymbol{b} \iff  L(U\boldsymbol{x}) = \boldsymbol{b} $$

Notice that the matrix-vector product $U\boldsymbol{x}$ is itself a vector, let's call it $\boldsymbol{c}$ for the time-being (i.e. 
$\boldsymbol{c}=U\boldsymbol{x}$).

The above system then reads 

$$ L\boldsymbol{c} = \boldsymbol{b} $$

where $L$ is a matrix and $\boldsymbol{c}$ is an unknown.  

As $L$ is in lower-triangular form we can use forward substitution (generalising the back subsitution algorithm/code we developed last week) to very easily find $\boldsymbol{c}$ in relatively few operations (we don't need to call the entire Gaussian elimination algorithm).

Once we know $\boldsymbol{c}$ we then solve the second linear system 

$$ U\boldsymbol{x} = \boldsymbol{c} $$

where now we can use the fact that $U$ is upper-triangular to use our back substitution algorithm again very efficiently to give the solution $\boldsymbol{x}$ we require.

So for a given $\boldsymbol{b}$ we can find the corresponding $\boldsymbol{x}$ very efficiently, we can therefore do this repeatedly as each new $\boldsymbol{b}$ is given to us.

Our challenge is therefore to find the matrices $L$ and $U$ that form the decomposition $A=LU$.


## LU decomposition - algorithm

Recall the comment above on the $L$ and $U$ matrices encoding the steps taken in Gaussian elimination.  Let's see how this works through the development of the so-called *Doolittle algorithm*.

Let's consider an example matrix:

$$
  A=\begin{bmatrix}
    {\color{black}5} & {\color{black}7} & {\color{black}5} & {\color{black}9}\\
{\color{black}5} & {\color{black}14} & {\color{black}7} & {\color{black}10}\\
{\color{black}20} & {\color{black}77} & {\color{black}41} & {\color{black}48}\\
{\color{black}25} & {\color{black}91} & {\color{black}55} & {\color{black}67}\\
    \end{bmatrix}
$$

the first step of Gaussian elimination is to set the
sub-diagonal elements in the first column to zero by subtracting multiples of
the first row from each of the subsequent rows. 

For this example, using the symbolic notiation from last week
this requires the row operations

\begin{align}
Eq. (2) &\leftarrow Eq. (2) - 1\times Eq. (1)\\
Eq. (3) &\leftarrow Eq. (3) - 4\times Eq. (1)\\
Eq. (4) &\leftarrow Eq. (4) - 5\times Eq. (1)\\
\end{align}

or mathematically, and for each element of the matrix (remembering that we are adding rows together - while one of
the entries of a row will end up being zero, this also has the consequence of updating the rest of the values in that row, hence the iteration over $j$ below):

\begin{align}
A_{2j} &\leftarrow A_{2j} - \frac{A_{21}}{A_{11}} A_{1j} = A_{2j} - \frac{5}{5} \times A_{1j} = A_{2j} \, {\color{Orange}{-1}} \times A_{1j}, \quad j=1,2,3,4\\
A_{3j} &\leftarrow A_{3j} - \frac{A_{31}}{A_{11}} A_{1j} = A_{3j} - \frac{20}{5} \times A_{1j} = A_{3j}\, {\color{Orange}{-4}} \times A_{1j} , \quad j=1,2,3,4\\
A_{4j} &\leftarrow A_{4j} - \frac{A_{41}}{A_{11}} A_{1j} = A_{4j} - \frac{25}{5} \times A_{1j}= A_{4j}\, {\color{Orange}{-5}} \times A_{1j}, \quad j=1,2,3,4\\
\end{align}

<br>

Notice that we can also write these exact operations on elements in terms of multiplication by a carefully chosen lower-triangular matrix where the non-zero's below the diagonal are restricted to a single column, e.g. for the example above

$$
  \begin{bmatrix}
    {\color{black}1} & {\color{black}0} & {\color{black}0} & {\color{black}0}\\
    {\color{Orange}{-1}} & {\color{black}1} & {\color{black}0} & {\color{black}0}\\
    {\color{Orange}{-4}} & {\color{black}0} & {\color{black}1} & {\color{black}0}\\
    {\color{Orange}{-5}} & {\color{black}0} & {\color{black}0} & {\color{black}1}\\   
  \end{bmatrix}\qquad\times\qquad\begin{bmatrix}
    {\color{black}5} & {\color{black}7} & {\color{black}5} & {\color{black}9}\\
    {\color{black}5} & {\color{black}14} & {\color{black}7} & {\color{black}10}\\
    {\color{black}20} & {\color{black}77} & {\color{black}41} & {\color{black}48}\\
    {\color{black}25} & {\color{black}91} & {\color{black}55} & {\color{black}67}\\   
  \end{bmatrix}\qquad=\qquad\begin{bmatrix}
    {\color{black}5} & {\color{black}7} & {\color{black}5} & {\color{black}9}\\
    {\color{blue}{0}} & {\color{blue}{7}} & {\color{blue}{2}} & {\color{blue}{1}}\\
    {\color{blue}{0}} & {\color{blue}{49}} & {\color{blue}{21}} & {\color{blue}{12}}\\
    {\color{blue}{0}} & {\color{blue}{56}} & {\color{blue}{30}} & {\color{blue}{22}}\\    
  \end{bmatrix}
$$

the blue portion of the matrix just highlights those entries that have been affected - notice that the first column of this submatrix is by construction all zero.

The lower-triangular matrix we pre-multiply by in the above (let's call this one $L_0$) is thus encoding the first step in Gaussian elimination.

<br>

The next step involves taking the second row of the updated matrix as the new pivot (we will ignore partial pivoting for simplicity), and subtracting multiples of this row from those below in order to set the zeros below the diagonal in the second column to zero. 

Note that this step can be achieved here with the multiplication by the following lower-triangular matrix (call this one $L_1$)

\begin{equation*}
  \begin{bmatrix}
    {\color{black}1} & {\color{black}0} & {\color{black}0} & {\color{black}0}\\
    {\color{black}0} & {\color{black}1} & {\color{black}0} & {\color{black}0}\\
    {\color{black}0} & {\color{Orange}{-7}} & {\color{black}1} & {\color{black}0}\\
    {\color{black}0} & {\color{Orange}{-8}} & {\color{black}0} & {\color{black}1}\\
  \end{bmatrix}\qquad\times\qquad\begin{bmatrix}
    {\color{black}5} & {\color{black}7} & {\color{black}5} & {\color{black}9}\\
    {\color{black}0} & {\color{black}7} & {\color{black}2} & {\color{black}1}\\
    {\color{black}0} & {\color{black}49} & {\color{black}21} & {\color{black}12}\\
    {\color{black}0} & {\color{black}56} & {\color{black}30} & {\color{black}22}\\
  \end{bmatrix}\qquad=\qquad\begin{bmatrix}
    {\color{black}5} & {\color{black}7} & {\color{black}5} & {\color{black}9}\\
    {\color{black}0} & {\color{black}7} & {\color{black}2} & {\color{black}1}\\
    {\color{black}0} & {\color{blue}{0}} & {\color{blue}{7}} & {\color{blue}{5}}\\
    {\color{black}0} & {\color{blue}{0}} & {\color{blue}{14}} & {\color{blue}{14}}\\
  \end{bmatrix}
\end{equation*}


<br>

And finally for this example to get rid the of the leading 14 on the final row:

\begin{equation*}
  \begin{bmatrix}
    {\color{black}1} & {\color{black}0} & {\color{black}0} & {\color{black}0}\\
    {\color{black}0} & {\color{black}1} & {\color{black}0} & {\color{black}0}\\
    {\color{black}0} & {\color{black}0} & {\color{black}1} & {\color{black}0}\\
    {\color{black}0} & {\color{black}0} & {\color{Orange}{-2}} & {\color{black}{1}}\\
  \end{bmatrix}\qquad\times\qquad\begin{bmatrix}
    {\color{black}5} & {\color{black}7} & {\color{black}5} & {\color{black}9}\\
    {\color{black}0} & {\color{black}7} & {\color{black}2} & {\color{black}1}\\
    {\color{black}0} & {\color{black}0} & {\color{black}7} & {\color{black}5}\\
    {\color{black}0} & {\color{black}0} & {\color{black}14} & {\color{black}14}\\
  \end{bmatrix}\qquad=\qquad\begin{bmatrix}
    {\color{black}5} & {\color{black}7} & {\color{black}5} & {\color{black}9}\\
    {\color{black}0} & {\color{black}7} & {\color{black}2} & {\color{black}1}\\
    {\color{black}0} & {\color{black}0} & {\color{black}7} & {\color{black}5}\\
    {\color{black}0} & {\color{black}0} & {\color{blue}{0}} & {\color{blue}{4}}\\
  \end{bmatrix}
\end{equation*}

where this lower triangular matrix we will call $L_2$, and the RHS matrix is now in upper-triangular form as we expect from Gaussian elimination (call this $U$).

In summary, the above operations (starting from $A$, first multiplying on the left by $L_0$, then multiplying the result on the left by $L_1$ and so on to arrive at an upper triangular matrix $U$) can be written as 

$$ L_2(L_1(L_0A)) = U $$


Note that these special lower-triangular matrices $L_0$, $L_1$ etc, are all examples of what is known as *atomic* lower-triangular matrices: a special form of unitriangular matrix (a triangular matrix with the diagonals all being unity)  where the off-diagonal entries are all zero apart from in a single column. 

Notice that for these special, simple matrices, their inverse is simply the original matrix but with the sign of those off-diagonals changed:

$$
\left[
  \begin{array}{rrrrrrrrr}
    1      & 0      & \cdots      &        &              &        &         &   & 0 \\
    0      & 1      & 0           & \cdots &              &        &         &   & 0 \\
    0      & \ddots & \ddots      & \ddots &              &        &         &   &  \vdots \\
    \vdots & \ddots & \ddots      & \ddots &              &        &         &   &  \\
           &        &             &   0    &   1          &  0     &         & &  &  \\           
           &        &             &   0    &   l_{i+1,i}  &  1     &  \ddots &   &  &  \\  
           &        &             &   0    &   l_{i+2,i}  &  0     &  \ddots &   & &  \\  
           &        &             & \vdots &   \vdots     & \vdots &  \ddots &   & 0 &  \\               
    0      & \cdots &             & 0      &   l_{n,i}    & 0      &  \cdots & 0 & 1 &  \\    
\end{array}
\right]^{-1}
=
\left[
  \begin{array}{rrrrrrrrr}
    1      & 0      & \cdots      &        &              &        &         &   & 0 \\
    0      & 1      & 0           & \cdots &              &        &         &   & 0 \\
    0      & \ddots & \ddots      & \ddots &              &        &         &   &  \vdots \\
    \vdots & \ddots & \ddots      & \ddots &              &        &         &   &  \\
           &        &             &   0    &   1          &  0     &         & &  &  \\           
           &        &             &   0    &   -l_{i+1,i}  &  1     &  \ddots &   &  &  \\  
           &        &             &   0    &   -l_{i+2,i}  &  0     &  \ddots &   & &  \\  
           &        &             & \vdots &   \vdots     & \vdots &  \ddots &   & 0 &  \\               
    0      & \cdots &             & 0      &   -l_{n,i}    & 0      &  \cdots & 0 & 1 &  \\    
\end{array}
\right]
$$

This means we can write down the product of their inverses trivially:

### <span style="color:blue">Exercise 1: lower-triangular matrices</span>

Convince yourselves of the following facts:


* The above statement on what the inverse of the special $L$ matrices is - check on a small example.


* The multiplication of arbitrary lower-triangular square matrices is also lower-triangular.


* $L_2(L_1(L_0A)) = U \implies A = L_0^{-1}(L_1^{-1}(L_2^{-1}U))$


* and hence that $A=LU$ where $U$ is the upper-triangular matrix found at the end of Gaussian elimination, and where $L$ is the following  matrix

$$ L = L_0^{-1}L_1^{-1}L_2^{-1} $$


* Finally, compute this product of these lower-triangular matrices to show that 

$$L = 
  \begin{bmatrix}
    {\color{black}1} & {\color{black}0} & {\color{black}0} & {\color{black}0}\\
    {\color{black}{1}} & {\color{black}1} & {\color{black}0} & {\color{black}0}\\
    {\color{black}{4}} & {\color{black}7} & {\color{black}1} & {\color{black}0}\\
    {\color{black}{5}} & {\color{black}8} & {\color{black}2} & {\color{black}1}\\   
  \end{bmatrix}
$$

i.e. that the multiplication of these individual atomic matrices (importantly in this order) simply merges the entries from the non-zero columns of each atomic matrix, and hence is both lower-triangular, as well as trivial to compute.

## LU decomposition - implementation

So we can build an LU code easily from our Gaussian elimination code from last lecture which already works out and performs these tasks.  

The final $U$ matrix we need here is as was already constructed through Gaussian elimination, the entries of $L$ we need are simply the ${A_{ik}}/{A_{kk}}$ multipliers we computed as part of the elimination, but threw away previously (we called them `s` in the code and computed them outside of the `j` loop).

<br>

***Implementation observation***

For a given pivot row $k$, for each of these multipliers (for every row below the pivot), as we compute them we know that we are going to transform the augmented matrix in order to achieve a new zero below the diagonal - this frees up space in the matrix that we could use to store each multiplier moving on to the following row, we implicitly know that the diagonals of $L$ will be unity and so don't need to store these (and noting that we don't actually have a space for them anyway!). We then move on to the following pivot row, replacing the zeros in the corresponding column we are zero'ing with the multipliers again.

For example, for the case above 

$$ A = 
  \begin{bmatrix}
    {\color{black}5} & {\color{black}7} & {\color{black}5} & {\color{black}9}\\
    {\color{black}5} & {\color{black}14} & {\color{black}7} & {\color{black}10}\\
    {\color{black}20} & {\color{black}77} & {\color{black}41} & {\color{black}48}\\
    {\color{black}25} & {\color{black}91} & {\color{black}55} & {\color{black}67}\\   
  \end{bmatrix}\quad\rightarrow\quad
  \begin{bmatrix}
    {\color{black}5} & {\color{black}7} & {\color{black}5} & {\color{black}9}\\
    {\color{blue}{1}} & {\color{black}{7}} & {\color{black}{2}} & {\color{black}{1}}\\
    {\color{blue}{4}} & {\color{black}{49}} & {\color{black}{21}} & {\color{black}{12}}\\
    {\color{blue}{5}} & {\color{black}{56}} & {\color{black}{30}} & {\color{black}{22}}\\    
  \end{bmatrix}\quad\rightarrow\quad
  \begin{bmatrix}
    {\color{black}5} & {\color{black}7} & {\color{black}5} & {\color{black}9}\\
    {\color{blue}1} & {\color{black}7} & {\color{black}2} & {\color{black}1}\\
    {\color{blue}4} & {\color{blue}{7}} & {\color{black}{7}} & {\color{black}{5}}\\
    {\color{blue}5} & {\color{blue}{8}} & {\color{black}{14}} & {\color{black}{14}}\\
  \end{bmatrix}\quad\rightarrow\quad
  \begin{bmatrix}
    {\color{black}5} & {\color{black}7} & {\color{black}5} & {\color{black}9}\\
    {\color{blue}1} & {\color{black}7} & {\color{black}2} & {\color{black}1}\\
    {\color{blue}4} & {\color{blue}7} & {\color{black}7} & {\color{black}5}\\
    {\color{blue}5} & {\color{blue}8} & {\color{blue}{2}} & {\color{black}{4}}\\
  \end{bmatrix}
  = [\color{blue}L\backslash U]
$$



### <span style="color:blue">Exercise 2: LU decomposition</span>

Starting from your Gaussian elimination code (or rather the `upper_triangle` part) produce a new code to compute the LU decomposition of a matrix. First, store $L$ and $U$ in two different matrices; you could then try a version where you store the entries of both $L$ and $U$ in $A$ as described above.

NB. In my sample solution I actually don't use the trick of using the zero'd column to store the multipliers, I create a new $L$ array to store this, and return this $L$ along with the upper triangular $U$ (which will be the transformed $A$ array) from the function.

## Partial pivoting

At the end of last week we commented that a problem would occur in a numerical implementation of our algorithm if we had a situation where the $A_{kk}$ we divide through by in the Gaussian elimination and/or back substitution algorithms was zero (or close to zero).

Using Gaussian elimination as an example, let's again consider the algorithm mid-way working on an arbitrary matrix system, i.e. assume that the first $k$ rows have already been transformed into upper-triangular form, while the equations/rows below are not yet in this form:

$$
\left[
  \begin{array}{rrrrrrr|r}
    A_{11} & A_{12} & A_{13} & \cdots & A_{1k}  & \cdots & A_{1n} & b_1 \\
    0      & A_{22} & A_{23} & \cdots & A_{2k} & \cdots & A_{2n} & b_2 \\
    0      & 0      & A_{33} & \cdots & A_{3k}  & \cdots & A_{3n} & b_3 \\
    \vdots & \vdots & \vdots & \ddots & \vdots  & \ddots & \vdots & \vdots \\
\hdashline    
    0      & 0      & 0      & \cdots & A_{kk}  & \cdots & A_{kn} & b_k \\    
    \vdots & \vdots & \vdots & \ddots & \vdots  & \ddots & \vdots & \vdots \\
    0      & 0      & 0      & \cdots & A_{nk}  & \cdots & A_{nn} & b_n \\
\end{array}
\right]
$$

Note we have drawn the horizontal dashed line one row higher than we did previously, as we are not going to simply assume without first checking that it is wise to take the current row $k$ as the pivot row, and $A_{kk}$ as the so-called pivot element.

We have a problem if $A_{kk}$ is zero as we can't use it to cancel entries below.

*Partial pivoting* is a *trick* that can be used to fix this problem. It works by using the fact that we can swap rows - we select the best pivot (row or element) as the one where the $A_{ik}$ (for $i\ge k$) value is largest (relative to the other values of components in its own row $i$), and then swaps this row with the current $k$ row.

To generalise the codes we have written so far we would simply need to search for this row, and perform the row swap operation.

Python's `scipy.linalg` library has its own implementation of the LU decomposition, and it uses partial pivoting by default - this is why it wasn't straightforward to compare our existing codes against the output of this function.

In [15]:
A = np.array([[ 5., 7.,   5.,  9.],
               [ 5., 14.,  7., 10.],
               [20., 77., 41., 48.],
               [25., 91. ,55., 67.]])
print(A)

[[ 5.  7.  5.  9.]
 [ 5. 14.  7. 10.]
 [20. 77. 41. 48.]
 [25. 91. 55. 67.]]


In [16]:
P, L , U = sl.lu(A)

# P here is a 'permutation matrix' that performs swaps based upon partial pivoting
print(P)  

# the fact that it isn't just the identity means that the scipy algorithm did indeed decide 
# to perform row swaps

[[0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]]


In [17]:
# the lower-triangular matrix
print(L)  

[[ 1.          0.          0.          0.        ]
 [ 0.2         1.          0.          0.        ]
 [ 0.8        -0.375       1.          0.        ]
 [ 0.2         0.375       0.33333333  1.        ]]


In [18]:
# the upper-triangular matrix
print(U)  

[[ 25.          91.          55.          67.        ]
 [  0.         -11.2         -6.          -4.4       ]
 [  0.           0.          -5.25        -7.25      ]
 [  0.           0.           0.           0.66666667]]


In [19]:
# double check that P*L*U does indeed equal A
print(P @ L @ U)
print(np.allclose(A, P@L@U))

[[ 5.  7.  5.  9.]
 [ 5. 14.  7. 10.]
 [20. 77. 41. 48.]
 [25. 91. 55. 67.]]
True


By looking at the form of $P$ above, we can actually re-order the rows of $A$ in advance and consider the LU decomposition of the matrix where $P=I$, as below. 

What I mean here is that since $A = PLU$, and as $P^{-1}=P^T$, that $P^T A = LU$, so we should reorder $A$ based on what pre-multiplication by the transpose of $P$ does.

As we haven't bothered implementing pivoting ourselves yet in our own codes, we can check that our LU implementations (without partial pivoting) recreates the $A$, $L$ and $U$ below after we've done this pre-re-ordering of $A$:

In [20]:
# original A
#A = np.array([[ 5., 7.,   5.,  9.],
#               [ 5., 14.,  7., 10.],
#               [20., 77., 41., 48.],
#               [25., 91. ,55., 67.]])

# a new A with swapped rows based on observation of P
A = np.array([[25. ,91. ,55. ,67.],
               [ 5.,  7.,  5.,  9.], 
               [20., 77., 41., 48.],
               [ 5., 14., 7.,  10.]])
print(A)

[[25. 91. 55. 67.]
 [ 5.  7.  5.  9.]
 [20. 77. 41. 48.]
 [ 5. 14.  7. 10.]]


In [21]:
# this is where this comes from
A = np.array([[ 5., 7.,   5.,  9.],
               [ 5., 14.,  7., 10.],
               [20., 77., 41., 48.],
               [25., 91. ,55., 67.]])
print(P.T@A)

# we use the transpose of P since A = PLU -> P^T A = LU

[[25. 91. 55. 67.]
 [ 5.  7.  5.  9.]
 [20. 77. 41. 48.]
 [ 5. 14.  7. 10.]]


In [22]:
# a new A with swapped rows based on observation of P
A = np.array([[25. ,91. ,55. ,67.],
               [ 5.,  7.,  5.,  9.], 
               [20., 77., 41., 48.],
               [ 5., 14., 7.,  10.]])


P, L, U = sl.lu(A)
# P now should be the identity matrix as pivoting no longer actually actions any row swaps with this A
print(P)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [23]:
# so this is the L we can compare our non-PP based implemetation against if we test it on this A
print(L)

[[ 1.          0.          0.          0.        ]
 [ 0.2         1.          0.          0.        ]
 [ 0.8        -0.375       1.          0.        ]
 [ 0.2         0.375       0.33333333  1.        ]]


In [24]:
# and the corresponding U
print(U)

[[ 25.          91.          55.          67.        ]
 [  0.         -11.2         -6.          -4.4       ]
 [  0.           0.          -5.25        -7.25      ]
 [  0.           0.           0.           0.66666667]]


In [25]:
print(P @ L @ U)

[[25. 91. 55. 67.]
 [ 5.  7.  5.  9.]
 [20. 77. 41. 48.]
 [ 5. 14.  7. 10.]]


### <span style="color:blue">Optional Exercise 3: Partial pivoting</span>

Implement partial pivoting.

## Ill-conditioned matrices

The conditioning (or lack of, i.e. the ill-conditioning) of matrices we are trying to invert - to obtain the inverse, or to find the solution to a corresponding linear matrix system - is incredibly important for the success of any algorithm.

When we started talking about matrices we noted that as long as the matrix is non-singular, i.e. $\det(A)\ne 0$, then an inverse exists, and a linear system with that $A$ has a unique solution.

But what happens when we consider a matrix that is *nearly* singular, i.e. $\det(A)$ is very small?

Well smallness is a relative term and so we need to ask the question of how large or small $\det(A)$ is compared to something.

That something is the *norm* of the matrix.


#### Vector norms

For a vector $\boldsymbol{v}$ (assumed to be an $n\times 1$ column vector) we have multiple possible norms to help us decide quantify the magnitude or size of that vector:

\begin{align}
\|\boldsymbol{v}\|_2 & = \sqrt{v_1^2 + v_2^2 + \cdots + v_n^2} = \left(\sum_{i=1}^n v_i^2 \right)^{1/2}, &\quad{\textrm{termed the two-norm or Euclidean norm}}\\
\|\boldsymbol{v}\|_1  & = |v_1| + |v_2| + \cdots + |v_n| = \sum_{i=1}^n |v_i|, &\quad{\textrm{termed the one-norm or taxi-cab norm}}\\
\|\boldsymbol{v}\|_{\infty}  &= \max\{|v_1|,|v_2|, \ldots, |v_n|\} = \max_{i=1}^n |v_i|, &\quad{\textrm{termed the max-norm or infinity norm}}
\end{align}


#### Matrix norms

We can define similar measures for the size of matrices, e.g. for $A$ which for complete generality we will assume is of shape $m\times n$:

\begin{align}
\|A\|_F & = \left(\sum_{i=1}^m \sum_{j=1}^n A_{ij}^2 \right)^{1/2}, &\quad{\textrm{termed the matrix Euclidean or Frobenius norm}}\\
\|A\|_{\infty} & = \max_{i=1}^m \sum_{j=1}^n|A_{i,j}|, &\quad{\textrm{termed the maximum absolute row-sum norm}}\\
\end{align}

Note that while these norms give different results (in both the vector and matrix cases), they are consistent or equivalent in that they are always within a constant factor of one another (a result that is true for finite-dimensional or discrete problems as we are considering here). 

This means we don't really need to worry too much about which norm we're using, as long as we are comparing vectors or matrices of the same size.

[NB. if $n$ or $m$ changes for example and we want to fairly compare two vectors or matrices, then in the vector case the so-called *RMS* error would in many cases be a more suitable choice than the two-norm.]

Let's evaluate some examples.

In [ ]:
A = np.array([[10., 2., 1.],[6., 5., 4.],[1., 4., 7.]])
print(A)

[[10.  2.  1.]
 [ 6.  5.  4.]
 [ 1.  4.  7.]]


In [ ]:
print(sl.norm(A))

15.748015748023622


In [ ]:
# the "Frobenius" (or Euclidean) norm from above 
print(sl.norm(A,'fro'))
# clearly the default for sl.norm as it returns the same answer as not specifying the norm
print(sl.norm(A,'fro') == sl.norm(A))

15.748015748023622
True


In [ ]:
# the maximum absolute row-sum
print(sl.norm(A,np.inf))

15.0


In [ ]:
# the maximum absolute column-sum
print(sl.norm(A,1))

17.0


In [ ]:
# the two-norm - note that this is NOT the same as the Frobenius norm
# as might be expected by comparing the terminology for the vector and matrix norms
# the matrix two-norm is also termed the spectral norm
print(sl.norm(A,2))

13.793091098640064


In [ ]:
# which is defined as (!!!!)
print(np.sqrt(np.real((np.max(sl.eigvals( A.T @ A))))))
# i.e. involves the eigenvalues of the matrix A.T@A, 
# or the so-called "singular values" of A - see module on Geophysical Inversion

13.793091098640062


### <span style="color:blue">Exercise 4: matrix norms</span>

Write some code to explicitly compute the two matrix norms defined mathematically above (i.e. the Frobenius and the maximum absolute row-sum norms) and compare against the values found above using in-built scipy functions.

Also, based on the above code and comments, what is the mathematical definition of the 1-norm and the 2-norm?


### Matrix conditioning

The (ill-)conditioning of a matrix is measured with the matrix condition number:

$$\textrm{cond}(A) = \|A\|\|A^{-1}\|$$

If this is close to one then $A$ is termed *well-conditioned*; the value increases with the degree of *ill-conditioning*, reaching infinity for a singular matrix.

Let's evaluate the condition number for the matrix above.

In [ ]:
A = np.array([[10., 2., 1.],[6., 5., 4.],[1., 4., 7.]])

print(A)
print(np.linalg.cond(A))  # let's use the in-built condition number function
print(sl.norm(A,2)*sl.norm(sl.inv(A),2))  # so the default condition number uses the matrix two-norm
print(np.linalg.cond(A,'fro')) # this is how to use a different norm
print(sl.norm(A,'fro')*sl.norm(sl.inv(A),'fro'))

[[10.  2.  1.]
 [ 6.  5.  4.]
 [ 1.  4.  7.]]
10.713371881346792
10.713371881346786
12.463616561943589
12.463616561943587


The condition number is expensive to compute, and so in practice the relative size of the determinant of the matrix can be gauged based on the magnitude of the entries of the matrix.

#### Example

We know that a singular matrix does not result in a unique solution to its corresponding linear matrix system. But what are the consequences of near-singularity (ill-conditioning)?

Consider the following example


$$
\left(
  \begin{array}{cc}
    2 & 1 \\
    2 & 1 + \epsilon  \\
  \end{array}
\right)\left(
  \begin{array}{c}
    x \\
    y \\
  \end{array}
\right) = \left(
  \begin{array}{c}
    3 \\
    0 \\
  \end{array}
\right)
$$

Clearly when $\epsilon=0$ the two columns/rows are not linear independent, and hence the determinant of this matrix is zero, the condition number is infinite, and the linear system does not have a solution (as the two equations would be telling us the contradictory information that $2x+y$ is equal to 3 and is also equal to 0).

### <span style="color:blue">Exercise 5: Ill-conditioned matrix</span>

For the example above, consider a range of small values for $\epsilon$ and calculate the matrix determinant and condition number.

You should find for $\epsilon=0.001$ that $\det(A)=0.002$ (i.e. quite a lot smaller than the other coefficients in the matrix) and $\textrm{cond}(A)\approx 5000$.

Using `sl.inv(A) @ b` you should also be able to compute the solution $\boldsymbol{x}=(1501.5,-3000.)^T$.

What happens when you make a very small change to the coefficients of the matrix (e.g. set $\epsilon=0.002$)?

You should find that this change of just $0.1\%$ in one of the coefficients of the matrix (i.e. $1.001$ becoming $1.002$) results in a $100\%$ change in both components of the solution!

This is the consequence of the matrix being *ill-conditioned* - we should be careful about trusting the numerical solution to ill-conditioned problems.

A way to see this is to recognise that computers do not perform arithmetic exactly - they necessarily have to [truncate numbers](http://www.mathwords.com/t/truncating_a_number.htm) at a certain number of significant figures, performing multiple operations with these truncated numbers can lead to an erosion of accuracy. Often this is not a problem, but these so-called [roundoff](http://mathworld.wolfram.com/RoundoffError.html) errors in algorithms generating $A$, or operating on $A$ as in Gaussian elimination etc, will lead to small inaccuracies in the coefficients of the matrix. Hence, in the case of ill-conditioned problems, will fall foul of the issue seen above where a very small error in an input to the algorithm led to a far larger error in an output.

### Roundoff errors

Note that in this course we have largely ignored the limitations of the floating point arithmetic performed by computers, including round-off errors.  

This is often the topic of the first lecture of courses, or first chapter of books, on Numerical Methods or Numerical Analysis - do take a look at some examples if you are interested.  

Also take a look at *D. Goldberg 1991: What every computer scientist should know about floating-point arithmetic, ACM Computing Surveys 23, Pages 5-48*.

For some examples of catastrophic failures due to round off errors see <https://www.ma.utexas.edu/users/arbogast/misc/disasters.html> and <http://ta.twi.tudelft.nl/users/vuik/wi211/disasters.html> and [the sinking of the Sleipner A offshore platform](http://www.ima.umn.edu/~arnold/disasters/sleipner.html).

<br>

As an example, consider the mathematical formula

$$f(x)=(1-x)^{10}.$$

We can of course relatively easily expand this out by hand

$$f(x)=1- 10x + 45x^2 - 120x^3 + 210x^4 - 252x^5 + 210x^6 - 120x^7 + 45x^8 - 10x^9 + x^{10}.$$

Mathematically these two expressions for $f(x)$ are identical; when evaluated by a computer different operations will be performed, which (we hope) should give the same answer. For numbers $x$ away from $1$ these two expressions do return (pretty much) the same answer.  

However, for $x$ close to 1 the second expression adds and subtracts individual terms of increasing size which should largely cancel out, but they don't to sufficient accuracy due to round off errors; these errors accumulate with more and more operations, leading a loss of significance <https://en.wikipedia.org/wiki/Loss_of_significance>

In [ ]:
def f1(x):
    return (1. - x)**10

def f2(x):
    return (1. - 10.*x + 45.*x**2 - 120.*x**3 +
           210.*x**4 - 252.*x**5 + 210.*x**6 -
           120.*x**7 + 45.*x**8 - 10.*x**9 + x**10)

# for a value of x away from 1
x=0.6
print(f1(x), f2(x), 1.-f1(x)/f2(x), '\n') # print values computed in different ways and their relative difference
# the error is far down the significant figures

# things get a bit worse as x gets closer to 1
x=0.8
print(f1(x), f2(x), 1.-f1(x)/f2(x), '\n') 

# and a significant error (21%) can be see for a number close to 1
x=0.95
print(f1(x), f2(x), 1.-f1(x)/f2(x)) 
# f2 is simply swamped with round off errors

0.00010485760000000006 0.00010485760000436464 4.1623815505431594e-11 

1.0239999999999978e-07 1.0240001356576212e-07 1.3247813024364063e-07 

9.765625000000086e-14 1.2378986724570495e-13 0.21111273343425307


### Algorithm stability

The susceptibility for a numerical algorithm to dampen (inevitable) errors, rather than to magnify them as we have seen in examples above, is termed *stability*.  This is a concern for numerical linear algebra as considered here, as well as for the numerical solution of differential equations.  In that case you don't want small errors to grow and accumulate as you propagate the solution to an ODE or PDE forward in time say.

If your algorithm is not inherently stable, or has other limitation, you need to understand and appreciate this, as it can cause catastrophic failures! 
